In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('combine_data_clean.csv')
df.head()

,id,title,userId,rating
0,862,Toy Story,1923,3.0
1,862,Toy Story,2103,5.0
2,862,Toy Story,5380,1.0
3,862,Toy Story,6177,4.0
4,862,Toy Story,6525,4.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   id      1048575 non-null  int64  
 1   title   1048575 non-null  object 
 2   userId  1048575 non-null  int64  
 3   rating  1048575 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 32.0+ MB


In [6]:
df["average"]=df.groupby('id').mean().reset_index()

ValueError: Wrong number of items passed 3, placement implies 1

In [6]:
df2=df.groupby('id')['rating'].nunique().reset_index()
df2

,id,rating
0,5,10
1,6,10
2,11,10
3,13,10
4,15,10
...,...,...
279,124837,2
280,136311,5
281,144982,10
282,151489,2


In [12]:
dfx= pd.merge(df1, df2, on='id', how='outer')
dfx= dfx.rename(columns={'rating_x': 'average', 'rating_y': 'count'})
dfx

,id,average,count
0,5,3.079565,10
1,6,3.841764,10
2,11,3.660591,10
3,13,3.326442,10
4,15,2.727040,10
...,...,...,...
279,124837,3.500000,2
280,136311,3.590909,5
281,144982,2.346154,10
282,151489,3.000000,2


In [15]:
vote_counts = dfx[dfx['count'].notnull()]['count'].astype('int')
vote_averages = dfx[dfx['average'].notnull()]['average'].astype('int')
C = vote_averages.mean()
m = vote_counts.quantile(0.75)
def weighted_rating(data):
    v = data['count'] 
    R = data['average']
    return (v / (v + m) * R) + (m / (m + v) * C)

dfx['weighted_rating'] = dfx.apply(weighted_rating, axis=1)


In [16]:
dfx

,id,average,count,weighted_rating
0,5,3.079565,10,2.911261
1,6,3.841764,10,3.292361
2,11,3.660591,10,3.201774
3,13,3.326442,10,3.034700
4,15,2.727040,10,2.734999
...,...,...,...,...
279,124837,3.500000,2,2.869131
280,136311,3.590909,5,3.025608
281,144982,2.346154,10,2.544556
282,151489,3.000000,2,2.785798
